In [1]:
from model_builder.GIS import tinyGIS
TG = tinyGIS()
import pandas as pd
import numpy as np
from typing import Dict, Union
from itertools import product
import os
import xarray as xr
from easymore import Utility as esmrut


tinyGIS initiated


## Reading and merging the ntopo and other catchment attribute to create master attribute file

In [2]:
# read and load the ntopo
master = xr.open_dataset('./data/topology/West_ntopo.nc')

## add soil to master

In [3]:
# read the soil and add to master
soil = xr.open_dataset('./data/gis/modified_West_stats_soil_classes.nc')
soil = esmrut.subset(soil,
                     np.array(master['COMID'].values),
                     mapping = {'var_id':'n','dim_id':'n'})
soil = esmrut.reorder(soil,
                      np.array(master['COMID'].values),
                      mapping = {'var_id':'n','dim_id':'n'})


# pass soil
master['s'] = soil['s']

# soil fraction
master['soil_frac'] = soil['soil_frac']
master['soil_frac'].values = soil['soil_frac'].values

# soil majority
master['soil_majority'] = soil['soil_majority']
master['soil_majority'].values = soil['soil_majority'].values

master

<xarray.Dataset>
Dimensions:        (n: 453, s: 12)
Coordinates:
  * n              (n) int64 0 1 2 3 4 5 6 7 ... 445 446 447 448 449 450 451 452
  * s              (s) object '1 clay' '2 silty clay' ... '12 sand'
Data variables: (12/22)
    COMID          (n) int64 71032073 71032106 71030390 ... 71034975 71027942
    length         (n) float64 ...
    lengthdir      (n) float64 ...
    sinuosity      (n) float64 ...
    slope          (n) float64 ...
    uparea         (n) float64 ...
    ...             ...
    unitarea       (n) float64 ...
    latitude       (n) float64 ...
    longitude      (n) float64 ...
    width          (n) float64 ...
    soil_frac      (n, s) float64 0.0 0.0 1.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    soil_majority  (n) int64 3 3 3 8 3 3 3 3 3 3 3 3 ... 3 3 3 3 3 3 3 3 3 3 3 3

## add land cover to the master

In [4]:
# read the LULC and add to master
LULC = xr.open_dataset('./data/gis/modified_West_stats_NA_NALCMS_landcover_2020_30m.nc')
LULC = esmrut.subset(LULC,
                     np.array(master['COMID'].values),
                     mapping = {'var_id':'n','dim_id':'n'})
LULC = esmrut.reorder(LULC,
                      np.array(master['COMID'].values),
                      mapping = {'var_id':'n','dim_id':'n'})


# pass soil
master['l'] = LULC['l']

# soil fraction
master['LULC_frac'] = LULC['LULC_frac']
master['LULC_frac'].values = LULC['LULC_frac'].values

# soil majority
master['LULC_majority'] = LULC['LULC_majority']
master['LULC_majority'].values = LULC['LULC_majority'].values

master

<xarray.Dataset>
Dimensions:        (n: 453, s: 12, l: 19)
Coordinates:
  * n              (n) int64 0 1 2 3 4 5 6 7 ... 445 446 447 448 449 450 451 452
  * s              (s) object '1 clay' '2 silty clay' ... '12 sand'
  * l              (l) object '1 Temperate or sub-polar needleleaf forest' .....
Data variables: (12/24)
    COMID          (n) int64 71032073 71032106 71030390 ... 71034975 71027942
    length         (n) float64 ...
    lengthdir      (n) float64 ...
    sinuosity      (n) float64 ...
    slope          (n) float64 ...
    uparea         (n) float64 ...
    ...             ...
    longitude      (n) float64 ...
    width          (n) float64 ...
    soil_frac      (n, s) float64 0.0 0.0 1.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    soil_majority  (n) int64 3 3 3 8 3 3 3 3 3 3 3 3 ... 3 3 3 3 3 3 3 3 3 3 3 3
    LULC_frac      (n, l) float64 0.3887 0.0 0.0 0.0 0.0 ... 0.0 0.6692 0.0 0.0
    LULC_majority  (n) object '16' '16' '1' '16' '1' ... '15' '15' '15' '17'

## add combination to the master

In [5]:
# read the LULC and add to master
comb = xr.open_dataset('./data/gis/modified_West_Combination.nc')
comb = esmrut.subset(comb,
                     np.array(master['COMID'].values),
                     mapping = {'var_id':'n','dim_id':'n'})
comb = esmrut.reorder(comb,
                      np.array(master['COMID'].values),
                      mapping = {'var_id':'n','dim_id':'n'})

# pass soil
master['m'] = comb['m']

# combination fraction
master['comb_frac'] = comb['comb_frac']
master['comb_frac'].values = comb['comb_frac'].values

# combination majority
master['comb_majority'] = comb['comb_majority']
master['comb_majority'].values = comb['comb_majority'].values

# comb_soil
master['comb_soil'] = comb['comb_soil']
master['comb_soil'].values = comb['comb_soil'].values

# comb_LULC
master['comb_LULC'] = comb['comb_LULC']
master['comb_LULC'].values = comb['comb_LULC'].values

master

<xarray.Dataset>
Dimensions:        (n: 453, s: 12, l: 19, m: 15)
Coordinates:
  * n              (n) int64 0 1 2 3 4 5 6 7 ... 445 446 447 448 449 450 451 452
  * s              (s) object '1 clay' '2 silty clay' ... '12 sand'
  * l              (l) object '1 Temperate or sub-polar needleleaf forest' .....
  * m              (m) object 'comb_0001' 'comb_0002' ... 'comb_0015'
Data variables: (12/28)
    COMID          (n) int64 71032073 71032106 71030390 ... 71034975 71027942
    length         (n) float64 ...
    lengthdir      (n) float64 ...
    sinuosity      (n) float64 ...
    slope          (n) float64 ...
    uparea         (n) float64 ...
    ...             ...
    LULC_frac      (n, l) float64 0.3887 0.0 0.0 0.0 0.0 ... 0.0 0.6692 0.0 0.0
    LULC_majority  (n) object '16' '16' '1' '16' '1' ... '15' '15' '15' '17'
    comb_frac      (n, m) float64 0.3887 0.0 0.0 0.0 0.0774 ... 0.0 0.0 0.0 0.0
    comb_majority  (n) object '0008' '0008' '0001' ... '0007' '0007' '0009'
    comb_soil      (m) object '3' '3' '3' '3' '3' '3' ... '3' '8' '8' '8' '8'
    comb_LULC      (m) object '1' '5' '6' '8' '10' ... '19' '1' '10' '16' '19'

## add the elevation to master

In [6]:
# read the LULC and add to master
elev = xr.open_dataset('./data/gis/West_stats_elv.nc')
elev = esmrut.subset(elev,
                     np.array(master['COMID'].values),
                     mapping = {'var_id':'n','dim_id':'n'})
elev = esmrut.reorder(elev,
                      np.array(master['COMID'].values),
                      mapping = {'var_id':'n','dim_id':'n'})

# add elevation
master['mean_elev'] = elev['mean_elev']
master['mean_elev'].values = elev['mean_elev'].values

master

<xarray.Dataset>
Dimensions:        (n: 453, s: 12, l: 19, m: 15)
Coordinates:
  * n              (n) int64 0 1 2 3 4 5 6 7 ... 445 446 447 448 449 450 451 452
  * s              (s) object '1 clay' '2 silty clay' ... '12 sand'
  * l              (l) object '1 Temperate or sub-polar needleleaf forest' .....
  * m              (m) object 'comb_0001' 'comb_0002' ... 'comb_0015'
Data variables: (12/29)
    COMID          (n) int64 71032073 71032106 71030390 ... 71034975 71027942
    length         (n) float64 ...
    lengthdir      (n) float64 ...
    sinuosity      (n) float64 ...
    slope          (n) float64 ...
    uparea         (n) float64 ...
    ...             ...
    LULC_majority  (n) object '16' '16' '1' '16' '1' ... '15' '15' '15' '17'
    comb_frac      (n, m) float64 0.3887 0.0 0.0 0.0 0.0774 ... 0.0 0.0 0.0 0.0
    comb_majority  (n) object '0008' '0008' '0001' ... '0007' '0007' '0009'
    comb_soil      (m) object '3' '3' '3' '3' '3' '3' ... '3' '8' '8' '8' '8'
    comb_LULC      (m) object '1' '5' '6' '8' '10' ... '19' '1' '10' '16' '19'
    mean_elev      (n) float64 2.341e+03 2.552e+03 ... 1.092e+03 1.16e+03

In [7]:
master.to_netcdf('./data/gis/master.nc')
master

<xarray.Dataset>
Dimensions:        (n: 453, s: 12, l: 19, m: 15)
Coordinates:
  * n              (n) int64 0 1 2 3 4 5 6 7 ... 445 446 447 448 449 450 451 452
  * s              (s) object '1 clay' '2 silty clay' ... '12 sand'
  * l              (l) object '1 Temperate or sub-polar needleleaf forest' .....
  * m              (m) object 'comb_0001' 'comb_0002' ... 'comb_0015'
Data variables: (12/29)
    COMID          (n) int64 71032073 71032106 71030390 ... 71034975 71027942
    length         (n) float64 ...
    lengthdir      (n) float64 ...
    sinuosity      (n) float64 ...
    slope          (n) float64 ...
    uparea         (n) float64 ...
    ...             ...
    LULC_majority  (n) object '16' '16' '1' '16' '1' ... '15' '15' '15' '17'
    comb_frac      (n, m) float64 0.3887 0.0 0.0 0.0 0.0774 ... 0.0 0.0 0.0 0.0
    comb_majority  (n) object '0008' '0008' '0001' ... '0007' '0007' '0009'
    comb_soil      (m) object '3' '3' '3' '3' '3' '3' ... '3' '8' '8' '8' '8'
    comb_LULC      (m) object '1' '5' '6' '8' '10' ... '19' '1' '10' '16' '19'
    mean_elev      (n) float64 2.341e+03 2.552e+03 ... 1.092e+03 1.16e+03